In [1]:
import keras, random, numpy as np, matplotlib.pyplot as plt
from string import punctuation
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, InputLayer, Bidirectional, TimeDistributed, Activation

Using TensorFlow backend.


In [2]:
def chunk_seq(seq, chunk_len):
    chunked_seq = []
    for i in range(0, len(seq), chunk_len):
        chunked_seq.append(seq[i:i+chunk_len])
    return chunked_seq

def get_labels(seq):
    labels_seq = []
    seq = seq.split()
    for i in range(len(seq)):
        if ',' in seq[i]:
            labels_seq.append('<comma>')
        elif '.' in seq[i]:
            labels_seq.append('<period>')
        else:
            labels_seq.append('<na>')
    return labels_seq

def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)]) 
        token_sequences.append(token_sequence) 
    return token_sequences

In [3]:
tb = TensorBoard(log_dir='./tf_logs/', batch_size=64, write_graph=True, histogram_freq=0)
data = open('./data/processed/ted_data', 'r', encoding='utf-8').read()
len(data)

28218860

In [4]:
data = data.lower()
data_split = data.split('\n')
all_data = ' '.join(data_split)
words = all_data.split()

In [5]:
# Chunk sequence
x = chunk_seq(words, 15)
sequences = [' '.join(seq) for seq in x]

In [6]:
# Get sequence labels
process_labels = [get_labels(seq) for seq in sequences]
process_labels = [' '.join(seq) for seq in process_labels]

In [7]:
# Check number of sequences and labels
print('Number of sequences: \t{}'.format(len(sequences)))
print('Number of labels: \t{}'.format(len(process_labels)))

Number of sequences: 	335394
Number of labels: 	335394


In [8]:
y_labels = open('./pos_labels', 'r').read()
y_labels = y_labels.split('\n')
all_labels = ' '.join(y_labels)
labels_tag = all_labels.split()

In [9]:
# Build words vocab
words_in_vocab = Counter(words)
vocab = sorted(words_in_vocab, key=words_in_vocab.get, reverse=True)

# Skip most common word
vocab_to_int = {word: index for index, word in enumerate(vocab, 2)}
vocab_to_int['-PAD-'] = 0  # The special value used for padding
vocab_to_int['-OOV-'] = 1  # The special value used for OOVs
unique_vocab = len(vocab_to_int)
print('Number of unique words:', len(vocab_to_int))

Number of unique words: 181377


In [10]:
# Build labels vocab
labels_in_vocab = Counter(labels_tag)
labels_vocab = sorted(labels_in_vocab, key=labels_in_vocab.get, reverse=True)
label_to_int = {t: i for i, t in enumerate(labels_vocab, 1)}
label_to_int['-PAD-'] = 0  # The special value used to padding

# Check labels
no_classes = len(label_to_int)
print('Number of unique labels:', no_classes)
print(label_to_int)

Number of unique labels: 4
{'<na>': 1, '<comma>': 2, '<period>': 3, '-PAD-': 0}


In [11]:
# Tokenize input sequences
seq_int = []
for seq in sequences:
    seq_int.append([vocab_to_int[word] for word in seq.split()])
print('Sample sequence:', sequences[10])
print('Sample sequence:', seq_int[10])

# Tokenize output labels
lab_int = []
for lab in y_labels:
    lab_int.append([label_to_int[word] for word in lab.split()])
print('Sample label:', lab_int[10])

Sample sequence: in education, you're not asked.(laughter)and you're never asked back, curiously. that's strange to me. but
Sample sequence: [8, 1500, 110, 32, 75874, 110, 164, 311, 859, 75875, 61, 1381, 4, 309, 21]
Sample label: [1, 2, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 1]


In [12]:
# Check max seq length
seq_len = Counter([len(seq) for seq in seq_int])
max_seq_len = max(seq_len)
print("Maximum sequence length: {}".format(max(seq_len)))

encoded_labels = [to_categorical(i, num_classes=no_classes) for i in lab_int]
print(encoded_labels[10])

Maximum sequence length: 15
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [13]:
# Pad sequences to 15 or sequence length, post padding
features = np.zeros((len(seq_int), 15), dtype=int)

for i, row in enumerate(seq_int):
    features[i, :len(row)] = np.array(row)[:15]

# Check that all sequences at at length 5
assert len(features)==len(seq_int)
assert len(features[0])==15

In [14]:
train_test_split_frac = 0.8
split_index = int(0.8*len(features))

# Split data into training, validation, and test data (features and labels, x and y)
train_x, left_over_x = features[:split_index], features[split_index:]
train_y, left_over_y = encoded_labels[:split_index], encoded_labels[split_index:]

val_test_index = int(0.5*len(left_over_x))
print('Validation/Test amount: \t{}'.format(val_test_index))

val_x, test_x = left_over_x[:val_test_index], left_over_x[val_test_index:]
val_y, test_y = left_over_y[:val_test_index], left_over_y[val_test_index:]

## print out the shapes of your resultant feature data
print('Training Dataset: \t{}'.format(train_x.shape))
print('Validation Dataset: \t{}'.format(val_x.shape))
print('Testing Dataset: \t{}'.format(test_x.shape))

Validation/Test amount: 	33539
Training Dataset: 	(268315, 15)
Validation Dataset: 	(33539, 15)
Testing Dataset: 	(33540, 15)


In [15]:
model = Sequential()
# model.add(InputLayer(input_shape=(max_seq_len,)))
model.add(Embedding(input_dim=unique_vocab, output_dim=128, input_length=max_seq_len))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(no_classes, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy', ignore_class_accuracy(0)])
model.summary()
model.fit(x=train_x, y=np.array(train_y), batch_size=64, epochs=2, validation_data=(val_x, np.array(val_y)),
          shuffle=True, verbose=1, callbacks=[tb])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 128)           23216256  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 15, 512)           788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 4)             2052      
Total params: 24,006,788
Trainable params: 24,006,788
Non-trainable params: 0
_________________________________________________________________
Train on 268315 samples, validate on 33539 samples
Epoch 1/2
268315/268315 [==============================] - 245s 915us/step - loss: 0.0544 - acc: 0.9785 - ignore_accuracy: 0.9786 - val_loss: 0.0287 - val_acc: 0.9870 - val_ignore_accuracy: 0.9870
Epoch 2/2
268315/268315 [==============================] - 245s 912us/step - loss: 0.0013 - acc: 0.9995 - ignore_accuracy: 0.9995 - val_loss: 0.03

In [16]:
# scores = model.evaluate(x=test_x, y=np.array(test_y), verbose=1)
# print('Accuracy: {}'.format(scores[1] * 100))

In [17]:
pred = model.predict(test_x)
print(test_x[0])
print(pred[0])

[  78   12   27    3  102 1578   64   28  120  328 1418   30   42  756
    4]
[[1.24522703e-08 9.99999762e-01 2.59608925e-07 1.13188803e-09]
 [1.12887697e-07 9.99997735e-01 1.11772795e-06 1.02823708e-06]
 [4.48813609e-08 9.99997735e-01 1.83559882e-06 3.79251588e-07]
 [4.36679226e-09 9.99999881e-01 7.78676181e-08 1.47648080e-08]
 [1.14226841e-08 9.99999881e-01 3.88848811e-08 2.28565380e-08]
 [3.14043431e-12 1.10060772e-09 1.00000000e+00 4.24195470e-08]
 [5.87399862e-09 1.00000000e+00 4.38118342e-09 3.88034188e-10]
 [1.01635784e-08 1.00000000e+00 2.57367585e-08 2.81074697e-09]
 [1.13985914e-08 9.99999881e-01 1.25559040e-07 1.12288525e-08]
 [1.68999978e-08 9.99999881e-01 6.33630606e-08 1.99309227e-08]
 [6.29304031e-09 1.00000000e+00 1.92707201e-08 4.29327374e-09]
 [1.43760097e-08 9.99999881e-01 1.18626296e-07 2.66695039e-08]
 [4.13458494e-08 9.99999523e-01 3.28858079e-07 9.41037612e-08]
 [1.65243552e-08 9.99999881e-01 5.85385749e-08 1.31199496e-08]
 [4.84594338e-08 1.00000000e+00 3.388308

In [18]:
# Sequence to predict
pred_x_seq = []
for x in test_x[400]:
    for i, v in vocab_to_int.items():
        if v == x:
            pred_x_seq.append(i)
print(' '.join(pred_x_seq))

# Predicted output
pred_y = pred[400].argmax(axis=1)
pred_y_seq = []
for x in pred_y:
    for i, v in label_to_int.items():
        if v == x:
            pred_y_seq.append(i)
print(' '.join(pred_y_seq))

cents on the dollar comes from. but in the federal government, where salaries are pinned
<na> <na> <na> <na> <na> <period> <na> <na> <na> <na> <comma> <na> <na> <na> <na>
